In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
247,AFG,Asia,Afghanistan,2020-09-03,38243.0,38.0,16.714,1409.0,3.0,1.143,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
426,ALB,Europe,Albania,2020-09-03,9728.0,122.0,114.429,296.0,6.0,4.714,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
674,DZA,Africa,Algeria,2020-09-03,45158.0,325.0,362.714,1525.0,7.0,8.571,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-09-02,France,288754.0,0.0,30704.0,0.0,0.0,0.0
0,2020-09-02,China,89959.0,0.0,4730.0,0.0,0.0,0.0
0,2020-09-02,Italy,271129.0,0.0,35517.0,0.0,0.0,0.0
0,2020-09-02,Spain,485851.0,0.0,29226.0,0.0,0.0,0.0
0,2020-09-02,United States,6157729.0,0.0,185561.0,0.0,0.0,0.0
0,2020-09-02,World,26294171.0,0.0,865149.0,0.0,0.0,0.0
0,2020-09-02,United Kingdom,338615.0,0.0,41542.0,0.0,0.0,0.0
0,2020-09-02,Germany,245982.0,0.0,9322.0,0.0,0.0,0.0
0,2020-09-02,Iran,381185.0,0.0,21928.0,0.0,0.0,0.0
0,2020-09-02,Turkey,273007.0,0.0,6452.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-09-03,France,288754.0,293024.0,30704.0,30686.0,-4270.0,18.0
0,2020-09-03,China,89959.0,89953.0,4730.0,4727.0,6.0,3.0
0,2020-09-03,Italy,271129.0,271515.0,35517.0,35497.0,-386.0,20.0
0,2020-09-02,Spain,485851.0,479554.0,29226.0,29194.0,6297.0,32.0
0,2020-09-03,United States,6157729.0,6114406.0,185561.0,185744.0,43323.0,-183.0
0,2020-09-03,World,26294171.0,26059065.0,865149.0,863535.0,235106.0,1614.0
0,2020-09-03,United Kingdom,338615.0,338676.0,41542.0,41514.0,-61.0,28.0
0,2020-09-03,Germany,245982.0,246116.0,9322.0,9321.0,-134.0,1.0
0,2020-09-03,Iran,381185.0,378752.0,21928.0,21797.0,2433.0,131.0
0,2020-09-03,Turkey,273007.0,273301.0,6452.0,6462.0,-294.0,-10.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")